В этом ноубуке используются обeченные тестовые модели, с их помощью происходит сэмплирование и далее закодированные категориальные переменные переводятся в изначальные. После чего замеряется качество.

Способ нормализации: standart scale

Добавление шума: ко всем признакам

In [ ]:
%cd ..

In [1]:
import json
import pandas as pd
import torch
import matplotlib.pyplot as plt

from utils import *
from models.tabddpm_ON_SC_NA.tabddpm_ON_SC_NA import *

from tqdm.notebook import tqdm

from eval.base_metrics import calculate_base_metrics
from eval.similarity import calculate_similarity
from eval.mle import calculate_mle
from eval.alpha_beta import calculate_alpha_beta
from eval.detection import calculate_detection
from eval.dcr import calculate_DCR

### Подготовка

In [2]:
dataname = 'adult_ON_SC_NA'
model_short = 'tabddpm_ON_SC_NA'

In [3]:
if torch.cuda.is_available():
    device = f'cuda'
else:
    device = 'cpu'
CONFIG.add_arg('device', device)
print(f"Device: {device}")

CONFIG.add_arg('dataname', dataname)
CONFIG.add_arg('real_path',
                   f'./synthetic/{dataname}/initial_real.csv')
CONFIG.add_arg('test_path',
                   f'./synthetic/{dataname}/initial_test.csv')

CONFIG.add_arg('info_path',
                   f'./data/{dataname}/initial_info.json')

Device: cuda


In [4]:
model_save_path_hist = []
for sigma in [0, 0.001, 0.01, 0.1, 0.25, 0.5]:
    model_save_path=f"./models/{model_short}/ckpt/{model_short}_const_{str(sigma).replace('.', '_')}"
    model_save_path_hist.append(model_save_path)

for i, sigmas in enumerate(range(10)):
    model_save_path=f"./models/{model_short}/ckpt/{model_short}_mult_{i}"
    model_save_path_hist.append(model_save_path)

### Sample + Eval

In [5]:
CONFIG.get_all_args()

{'dataname': 'adult_ON_SC_NA',
 'method': 'tabddpm_ON_SC_NA_const_0',
 'device': 'cuda',
 'mode': 'train',
 'train': 1,
 'sample_save_path': './synthetic/adult_ON_SC_NA/initial_tabddpm_ON_SC_NA_const_0.csv',
 'sigma_scheduller_name': 'constant',
 'sigma_value': 0.001,
 'num_noise': 103,
 'real_path': './synthetic/adult_ON_SC_NA/initial_real.csv',
 'test_path': './synthetic/adult_ON_SC_NA/initial_test.csv',
 'info_path': './data/adult_ON_SC_NA/initial_info.json',
 'save_path': './synthetic/adult_ON_SC_NA/initial_tabddpm_ON_SC_NA_const_0.csv'}

In [1]:
overall_metrics = {}

for model_save_path in model_save_path_hist:
    model_name = model_save_path.split('/')[-1]
    sample_save_path = f'./synthetic/{dataname}/{model_name}.csv'

    CONFIG.add_arg('method', model_name)
    CONFIG.add_arg('sample_save_path',
                       f"synthetic/{CONFIG.get_arg('dataname')}/{CONFIG.get_arg('method')}.csv")

    overall_metrics[model_name] = {}

    print(model_save_path, model_name, sample_save_path)

    tabddpm_noise_ohe = TabDDPM_OHE_Noise_ALL(CONFIG, model_save_path=model_save_path, sigmas=None)
    tabddpm_noise_ohe.sample(sample_save_path=sample_save_path)
    postsample_OHE(dataname, f'./synthetic/{dataname}/initial_{model_name}.csv')

    # подсчет метрик
    try:
        overall_metrics[model_name]['base_metrics'] = calculate_base_metrics(make_binary=True, value=' >50K') # ошибка
    except Exception as e:
        overall_metrics[model_name]['base_metrics'] = np.nan
    overall_metrics[model_name]['similarity'] = calculate_similarity()
    overall_metrics[model_name]['mle'] = calculate_mle()
    overall_metrics[model_name]['detection'] = calculate_detection()
    overall_metrics[model_name]['DCR'] = calculate_DCR()
    overall_metrics[model_name]['quality'] = calculate_alpha_beta()

In [ ]:
final_metrics_table = []
for m in overall_metrics.keys():
    # Сбор таблички результатов
    tmp = pd.DataFrame([{'Model':model_short, 'Type':m, 'Data':dataname}])
    tmp.columns = pd.MultiIndex.from_tuples([('', i) for i in tmp.columns])
    result = [tmp]
    
    for metric_group in overall_metrics[m].keys():
        tmp = pd.DataFrame([overall_metrics[m][metric_group]])
        tmp.columns = pd.MultiIndex.from_tuples([(metric_group, i) for i in tmp.columns])
        result.append(tmp)
    result = pd.concat(result, axis = 1)
    final_metrics_table.append(result)
    
final_metrics_table = pd.concat(final_metrics_table)

In [18]:
final_metrics_table.drop(columns=('base_metrics', 0))

\
        Model                          Type            Data   
0  TabDDPM ON      tabddpm_ON_SC_NA_const_0  adult_ON_SC_NA   
0  TabDDPM ON  tabddpm_ON_SC_NA_const_0_001  adult_ON_SC_NA   
0  TabDDPM ON   tabddpm_ON_SC_NA_const_0_01  adult_ON_SC_NA   
0  TabDDPM ON    tabddpm_ON_SC_NA_const_0_1  adult_ON_SC_NA   
0  TabDDPM ON   tabddpm_ON_SC_NA_const_0_25  adult_ON_SC_NA   
0  TabDDPM ON    tabddpm_ON_SC_NA_const_0_5  adult_ON_SC_NA   
0  TabDDPM ON       tabddpm_ON_SC_NA_mult_0  adult_ON_SC_NA   
0  TabDDPM ON       tabddpm_ON_SC_NA_mult_1  adult_ON_SC_NA   
0  TabDDPM ON       tabddpm_ON_SC_NA_mult_2  adult_ON_SC_NA   
0  TabDDPM ON       tabddpm_ON_SC_NA_mult_3  adult_ON_SC_NA   
0  TabDDPM ON       tabddpm_ON_SC_NA_mult_4  adult_ON_SC_NA   
0  TabDDPM ON       tabddpm_ON_SC_NA_mult_5  adult_ON_SC_NA   
0  TabDDPM ON       tabddpm_ON_SC_NA_mult_6  adult_ON_SC_NA   
0  TabDDPM ON       tabddpm_ON_SC_NA_mult_7  adult_ON_SC_NA   
0  TabDDPM ON       tabddpm_ON_SC_NA_mult_8  adult_ON_SC_NA   
0  TabDDPM ON       tabddpm_ON_SC_NA_mult_9  adult_ON_SC_NA   

              similarity                              \
  Column Shapes Score, % Column Pair Trends Score, %   
0              88.660053                   46.445030   
0              88.479469                   51.340843   
0              88.758740                   45.135087   
0              88.765087                   48.108640   
0              89.280632                   46.925033   
0              88.982935                   59.137415   
0              88.916188                   59.134907   
0              89.264457                   44.800411   
0              88.938505                   46.802076   
0              88.660053                   43.400733   
0              88.783924                   52.691277   
0              88.828967                   48.791950   
0              88.848418                   46.601659   
0              88.064453                   52.205256   
0              88.828353                   58.206029   
0              88.923968                   54.582627   

                              \
  Overall Score (Average), %   
0                  67.552542   
0                  69.910156   
0                  66.946913   
0                  68.436863   
0                  68.102833   
0                  74.060175   
0                  74.025548   
0                  67.032434   
0                  67.870291   
0                  66.030393   
0                  70.737600   
0                  68.810459   
0                  67.725039   
0                  70.134855   
0                  73.517191   
0                  71.753298   

                                                       \
  Error rate (%) of column-wise density estimation, %   
0                                           11.339947   
0                                           11.520531   
0                                           11.241260   
0                                           11.234913   
0                                           10.719368   
0                                           11.017065   
0                                           11.083812   
0                                           10.735543   
0                                           11.061495   
0                                           11.339947   
0                                           11.216076   
0                                           11.171033   
0                                           11.151582   
0                                           11.935547   
0                                           11.171647   
0                                           11.076032   

                                                           \
  Error rate (%) of column-wise density estimation std, %   
0                                               15.753769   
0                                               16.305656   
0                                               16.064552   
0       

In [16]:
pd.set_option('display.max_columns', None)
final_metrics_table.round(3)

base_metrics  \
        Model                          Type            Data            0   
0  TabDDPM ON      tabddpm_ON_SC_NA_const_0  adult_ON_SC_NA          NaN   
0  TabDDPM ON  tabddpm_ON_SC_NA_const_0_001  adult_ON_SC_NA          NaN   
0  TabDDPM ON   tabddpm_ON_SC_NA_const_0_01  adult_ON_SC_NA          NaN   
0  TabDDPM ON    tabddpm_ON_SC_NA_const_0_1  adult_ON_SC_NA          NaN   
0  TabDDPM ON   tabddpm_ON_SC_NA_const_0_25  adult_ON_SC_NA          NaN   
0  TabDDPM ON    tabddpm_ON_SC_NA_const_0_5  adult_ON_SC_NA          NaN   
0  TabDDPM ON       tabddpm_ON_SC_NA_mult_0  adult_ON_SC_NA          NaN   
0  TabDDPM ON       tabddpm_ON_SC_NA_mult_1  adult_ON_SC_NA          NaN   
0  TabDDPM ON       tabddpm_ON_SC_NA_mult_2  adult_ON_SC_NA          NaN   
0  TabDDPM ON       tabddpm_ON_SC_NA_mult_3  adult_ON_SC_NA          NaN   
0  TabDDPM ON       tabddpm_ON_SC_NA_mult_4  adult_ON_SC_NA          NaN   
0  TabDDPM ON       tabddpm_ON_SC_NA_mult_5  adult_ON_SC_NA          NaN   
0  TabDDPM ON       tabddpm_ON_SC_NA_mult_6  adult_ON_SC_NA          NaN   
0  TabDDPM ON       tabddpm_ON_SC_NA_mult_7  adult_ON_SC_NA          NaN   
0  TabDDPM ON       tabddpm_ON_SC_NA_mult_8  adult_ON_SC_NA          NaN   
0  TabDDPM ON       tabddpm_ON_SC_NA_mult_9  adult_ON_SC_NA          NaN   

              similarity                              \
  Column Shapes Score, % Column Pair Trends Score, %   
0                 88.660                      46.445   
0                 88.479                      51.341   
0                 88.759                      45.135   
0                 88.765                      48.109   
0                 89.281                      46.925   
0                 88.983                      59.137   
0                 88.916                      59.135   
0                 89.264                      44.800   
0                 88.939                      46.802   
0                 88.660                      43.401   
0                 88.784                      52.691   
0                 88.829                      48.792   
0                 88.848                      46.602   
0                 88.064                      52.205   
0                 88.828                      58.206   
0                 88.924                      54.583   

                              \
  Overall Score (Average), %   
0                     67.553   
0                     69.910   
0                     66.947   
0                     68.437   
0                     68.103   
0                     74.060   
0                     74.026   
0                     67.032   
0                     67.870   
0                     66.030   
0                     70.738   
0                     68.810   
0                     67.725   
0                     70.135   
0                     73.517   
0                     71.753   

                                                       \
  Error rate (%) of column-wise density estimation, %   
0                                              11.340   
0                                              11.521   
0                                              11.241   
0                                              11.235   
0                                              10.719   
0                                              11.017   
0                                              11.084   
0                                              10.736   
0                                              11.061   
0                                              11.340   
0                                              11.216   
0                                              11.171   
0                                              11.152   
0                                              11.936   
0                                              11.172   
0                                              11.076   

                                                           \
  Error rate (%) 

In [21]:
import os
if not os.path.exists(f'./eval/total/{dataname}'):
    os.makedirs(f'./eval/total/{dataname}')
final_metrics_table.to_csv(f'./eval/total/{dataname}/{model_short}_final_metrcs_table.csv', index=False)